In [1]:
env KERAS_BACKEND=tensorflow

env: KERAS_BACKEND=tensorflow


In [2]:
!pip install numpy==1.16.2



In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [4]:
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [0]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)

最大長度改成192

In [0]:
x_train = sequence.pad_sequences(x_train, maxlen = 192)
x_test = sequence.pad_sequences(x_test, maxlen = 192)

試了許多N, K 發現不管怎麼取，大概的acc都可以到80+，但val_acc都沒有超過87%

後來想想為什麼N=3會works，想說就大概把字分為: 好評，不影響，差評，3類。  
而這裡我們加強分類，把字分為: 極好評，好評，不影響，差評，極差評 5類，所以N = 5  
  
  而K就稍微取大一點


In [0]:
N = 5 
K = 8

觀摩其他大大，也加入Dropout層

In [8]:
model = Sequential()

model.add(Embedding(10000, N))
model.add(Dropout(0.5))

model.add(LSTM(K))
model.add(Dropout(0.28))

model.add(Dense(1, activation='sigmoid'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model.compile(loss = 'binary_crossentropy',
             optimizer = 'Adam',
             metrics=['acc'])

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 5)           50000     
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 5)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 8)                 448       
_________________________________________________________________
dropout_2 (Dropout)          (None, 8)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 9         
Total params: 50,457
Trainable params: 50,457
Non-trainable params: 0
_________________________________________________________________


In [0]:
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [12]:
model_his = model.fit(x_train, y_train,
                   batch_size=32,
                   epochs=8,
                   verbose=1, 
                   validation_data=(x_test, y_test),
                   callbacks=callbacks_list)

Instructions for updating:
Use tf.cast instead.
Train on 25000 samples, validate on 25000 samples
Epoch 1/8
25000/25000 [==============================] - 79s 3ms/step - loss: 0.5677 - acc: 0.7099 - val_loss: 0.4144 - val_acc: 0.8152

Epoch 00001: val_acc improved from -inf to 0.81524, saving model to weights-improvement-01-0.82.hdf5
Epoch 2/8
25000/25000 [==============================] - 77s 3ms/step - loss: 0.3544 - acc: 0.8596 - val_loss: 0.3062 - val_acc: 0.8728

Epoch 00002: val_acc improved from 0.81524 to 0.87280, saving model to weights-improvement-02-0.87.hdf5
Epoch 3/8
25000/25000 [==============================] - 77s 3ms/step - loss: 0.2866 - acc: 0.8919 - val_loss: 0.2962 - val_acc: 0.8759

Epoch 00003: val_acc improved from 0.87280 to 0.87588, saving model to weights-improvement-03-0.88.hdf5
Epoch 4/8
25000/25000 [==============================] - 76s 3ms/step - loss: 0.2542 - acc: 0.9054 - val_loss: 0.3066 - val_acc: 0.8686

Epoch 00004: val_acc did not improve from 0.8

最後取最高的val_acc達到87.588%